In [1]:
import sys
import os
import pandas as pd
import json
import numpy as np
import pickle
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer
from datasets import load_from_disk, load_metric, Dataset, load_dataset

#from captum.concept import TCAV, Concept
from captum.concept._utils.data_iterator import dataset_to_dataloader, CustomIterableDataset


In [15]:
A = {}
A['taregt'] ={ 'c': {'lay': {'tcav':0}}, 'r':{'l':0}}


In [16]:
A

{'taregt': {'c': {'lay': {'tcav': 0}}, 'r': {'l': 0}}}

In [20]:
A['taregt']['c']['lay'] = {'tcav':[1,2,3,4,4], 'acc' :123}
A['taregt']['c']['lay2'] = {'tcav':[1,2,3,4,4], 'acc' :123}

In [21]:
A

{'taregt': {'c': {'lay': {'tcav': [1, 2, 3, 4, 4], 'acc': 123},
   'lay2': {'tcav': [1, 2, 3, 4, 4], 'acc': 123}},
  'r': {'l': 0}}}

In [14]:
A

{'taregt': {'c': {'cc': [1, 2, 3, 4, 4], 'bb': 123}, 't': {'a': 0}}}

In [21]:
# CAVS Random
PATH_TO_Data = '/work3/s174498/concept_random_dataset/'

PATH_random_cav = PATH_TO_Data+'cavs/'+'linear'+ '_classifier_on_layer_' + str(0)+'_random.pkl'

with open(PATH_random_cav,'rb') as handle:
    data = pickle.load(handle)
    cav_random = data['cavs']
    acc_random = data['acc']

In [32]:
num_random_set = 100 #num_random_set
counter_set='wikipedia_split'
PATH_TO_Data = '/work3/s174498/concept_random_dataset/'


num_ex_in_set = 150
Data = counter_set #'wikipedia_split'

file_name =  f'tensor_{Data}_on_{0}_layer_{num_random_set}_sets_with_{num_ex_in_set}' # f'tensor_{Data}_on_{layer_nr}_layer_{num_random_set}_sets_with_{num_ex_in_set}'
file_random = PATH_TO_Data + '/'+ Data + '/' + file_name + '.pt'

random_rep = torch.load(file_random)


In [45]:
N = int(len(random_rep)/100)
for i in range(98,100-1):
    print(len(random_rep[i*N:(i+1)*N]))
    
random_rep[i*N:(i+1)*N].shape


150


(150, 768)

In [35]:
random_rep[-1].shape

(768,)

In [25]:
len(cav_random)

100

In [13]:
PATH_grad = '/work3/s174498/sst2_dataset/grads_logits/'+'linear'+'_class_'+str(0)+'_layer_'+str(11)+'.pkl'


In [15]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [17]:
with open(PATH_grad,'rb') as handle:
    data = pickle.load(handle)
grads = data['grads']
logits = data['logits']

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [11]:
cav_random.shape

AttributeError: 'list' object has no attribute 'shape'

In [8]:
n = 0
for i in cav_random:
    n += 1
    print(n)
    print(len(i))

1
768
2
768
3
768
4
768
5
768
6
768
7
768
8
768
9
768
10
768
11
768
12
768
13
768
14
768
15
768
16
768
17
768
18
768
19
768
20
768
21
768
22
768
23
768
24
768
25
768
26
768
27
768
28
768
29
768
30
768
31
768
32
768
33
768
34
768
35
768
36
768
37
768
38
768
39
768
40
768
41
768
42
768
43
768
44
768
45
768
46
768
47
768
48
768
49
768
50
768
51
768
52
768
53
768
54
768
55
768
56
768
57
768
58
768
59
768
60
768
61
768
62
768
63
768
64
768
65
768
66
768
67
768
68
768
69
768
70
768
71
768
72
768
73
768
74
768
75
768
76
768
77
768
78
768
79
768
80
768
81
768
82
768
83
768
84
768
85
768
86
768
87
768
88
768
89
768
90
768
91
768
92
768
93
768
94
768
95
768
96
768
97
768
98
768
99
768
100
768
101
101


In [2]:
sys.path.insert(0,'/zhome/94/5/127021/speciale/master_project')
from src.models.captum.tcav import TCAV
from src.models.captum.concept import Concept

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
checkpoint = '/work3/s174498/final/linear_head/checkpoint-1500' #finetuning-sentiment-model-test-head3/checkpoint-500'

model = RobertaForSequenceClassification.from_pretrained(checkpoint,output_hidden_states = True,return_dict = True,)
#model.eval()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /work3/s174498/final/linear_head/checkpoint-1500 and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# tokenizer
tokenizer_checkpoint = RobertaTokenizer.from_pretrained(checkpoint) 
#tokenizer_pretrained = RobertaTokenizer.from_pretrained('roberta-base')

In [6]:
# load
datadir = '/work3/s174498/sst2_dataset/'
#test_0_dataset = load_from_disk(datadir + 'test_0_dataset')
test_1_dataset = load_from_disk(datadir + 'test_1_dataset')

In [7]:
#tokenizer_checkpoint.encode(test_0_dataset['sentence'][0])

In [8]:
#model.roberta.encoder.layer[0].attention

In [9]:
N = 200
#for i in range(len(test_1_dataset['sentence'])):
#    j = len(test_1_dataset['sentence'][i])
#    if j > N :
#        N = j



In [10]:
def get_tensor_from_filename(filename):
    #datadir = '/work3/s174498/sst2_dataset/'
    ds = load_from_disk(filename)
    const_len = N
    ds_text = ds['sentence']

    for i in range(len(ds_text)):
        #text_idx = tokenizer_checkpoint.tokenize(ds_text[i])
        #text_idx[i] = tokenizer_checkpoint.encode(text_idx[i])
        text_idx = tokenizer_checkpoint.encode(ds_text[i])
        text_idx += [1] * max(0, const_len - len(text_idx[i]))
        text_indices = torch.tensor(text_idx, device=device).unsqueeze(0)
        yield text_indices

## Hent concept data

In [11]:
def assemble_concept(name, id, concepts_path="/work3/s174498/sst2_dataset/"):
    concept_path = os.path.join(concepts_path, name) + "/"
    #print(concept_path)
    dataset = CustomIterableDataset(get_tensor_from_filename, concept_path)
    #print(dataset.)
    concept_iter = dataset_to_dataloader(dataset, batch_size=1)
    #print(concept_iter)

    return Concept(id=id, name=name, data_iter=concept_iter)

In [12]:
pos_concept = assemble_concept("positive", 0, concepts_path="/work3/s174498/sst2_dataset/")
#pos_concept2 = assemble_concept("positive", 2, concepts_path="/work3/s174498/sst2_dataset/")

neg_concept = assemble_concept("negative", 5, concepts_path="/work3/s174498/sst2_dataset/")

In [13]:
experimental_sets=[[pos_concept, neg_concept]]

### Input target text
**Should be converted to tensor**

In [14]:
input_text = test_1_dataset['sentence'][:10]

In [15]:
def covert_text_to_tensor(input_texts):
    input_tensors = []
    for input_text in input_texts:
        const_len = N
        #text_idx = tokenizer_checkpoint.tokenize(input_text)
        text_idx = tokenizer_checkpoint.encode(input_text)
        text_idx += [1] * max(0, const_len - len(text_idx))
        input_tensor = torch.tensor(text_idx, device=device).unsqueeze(0)
        input_tensors.append(input_tensor)
    return torch.cat(input_tensors)

input_text_indices = covert_text_to_tensor(input_text)


In [16]:
tcav = TCAV(model, layers=['roberta.encoder.layer.0.output.LayerNorm'])

/zhome/94/5/127021/miniconda3/envs/roberta_env/lib/python3.9/site-packages/captum/concept/_utils/classifier.py:130: UserWarning: Using default classifier for TCAV which keeps input both train and test datasets in the memory. Consider defining your own classifier that doesn't rely heavily on memory, for large number of concepts, by extending `Classifer` abstract class
  warnings.warn(


In [17]:
tcav.interpret(input_text_indices , experimental_sets=experimental_sets)

>>> TEST 1 <<<
>>> CAVs 1
>>> CAVs 2.0
>>> CAVs 2.1
>>> gen.act. 1
>>> GEN ACT 1
>>> GEN ACT 2
>>> GEN ACT 3
>>> GEN ACT 4


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'generator'>

In [19]:
pos_concept.data_iter

TypeError: 'DataLoader' object is not subscriptable

In [ ]:
model.roberta.encoder#.layer[0].attention

: 

: 